In [ ]:
!pip install wandb
!wandb login

In [ ]:
LEARNING_RATES = [0.005, 0.003, 0.001, 0.007] # best = 0.003
# leaky_relu > ~ tanh, relu
# batch size = 128
# Batch norm is always essential
# one layer in encoder less is okay with more filters
# Reshaping to feed into Conv2DTranspose is better than using Dense then reshaping to feed into Conv2dTranspose. 

## Prelim stuff

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, Conv2DTranspose, Dense, Flatten, Reshape, BatchNormalization, Lambda, Add, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from PIL import Image, ImageOps
from skimage.metrics import structural_similarity as ssim

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import os

from tensorflow.keras.datasets import fashion_mnist, cifar10

In [ ]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
input_sizes_dict = {
    'm': 28,
    'c': 32,
}
channels_dict = {
    'm': 1,
    'c': 3,
}

In [ ]:
def fit_model_and_log_metrics(run_name, run_notes, model, epochs, batch_size):
    run = wandb.init(project="autoencoders_RQ_comparison", entity="csc2231-yash-siphelele", name=run_name, notes=run_notes)
    wandb_callback = WandbCallback(
                monitor="val_loss", verbose=0, mode="auto", save_weights_only=(False),
                log_weights=(False), log_gradients=(False), save_model=(True),
                training_data=None, validation_data=None, labels=[], predictions=36,
                generator=None, input_type=None, output_type=None, log_evaluation=(False),
                validation_steps=None, class_colors=None, log_batch_frequency=None,
                log_best_prefix="best_", save_graph=(False), validation_indexes=None,
                validation_row_processor=None, prediction_row_processor=None,
                infer_missing_processors=(False), log_evaluation_frequency=0)   
    early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=5, mode='auto')
    history = model.fit(x_train, x_train, epochs = epochs, batch_size = batch_size, validation_data = (x_test, x_test), callbacks=[wandb_callback, early_stopping_callback]) 
    decoded_imgs = model.predict(x_test)
    ssim_scores = []
    for i in range(len(x_test)):
        if CURRENT_DATASET_TYPE == 'c':
            ssim_value = ssim(x_test[i].reshape(input_sizes_dict[CURRENT_DATASET_TYPE], input_sizes_dict[CURRENT_DATASET_TYPE], channels_dict[CURRENT_DATASET_TYPE]), 
                                            decoded_imgs[i].reshape(input_sizes_dict[CURRENT_DATASET_TYPE], input_sizes_dict[CURRENT_DATASET_TYPE], channels_dict[CURRENT_DATASET_TYPE]),
                              multichannel=True)
        else:
            ssim_value = ssim(x_test[i].reshape(input_sizes_dict[CURRENT_DATASET_TYPE], input_sizes_dict[CURRENT_DATASET_TYPE]), 
                                            decoded_imgs[i].reshape(input_sizes_dict[CURRENT_DATASET_TYPE], input_sizes_dict[CURRENT_DATASET_TYPE]))
        ssim_scores.append(ssim_value)

    avg_ssim_score_x_test = np.mean(ssim_scores)
    wandb.log({"avg_ssim_score_x_test": avg_ssim_score_x_test})
    print("Avg SSIM over X_test:", avg_ssim_score_x_test)
    # model.save(os.path.join(wandb.run.dir, "model.h5"))
    run.finish()

In [ ]:
# (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
# x_train = x_train.astype('float32') / 255
# x_test = x_test.astype('float32') / 255
# x_train.shape, x_test.shape, y_train.shape, y_test.shape 

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_train.shape, x_test.shape, y_train.shape, y_test.shape 

170508288/170498071 [==============================] - 3s 0us/step


((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 1), (10000, 1))

In [ ]:
Z_DIM = [20, 100, 300, 5000]  # Dimension of the latent vector (z), model is trained using different values of Z_DIM

In [ ]:
CURRENT_DATASET_TYPE = 'c'

# Experiments

## Exp 1

In [ ]:
'''
Model recons_1
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='leaky_relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='leaky_relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z = Dense(latent_dim)(x)

encoder = Model(input_img,z)

# encoder.summary()

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='leaky_relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same', activation='leaky_relu')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)
# decoder.summary()
# apply the decoder to the sample from the latent distribution
pred = decoder(z)

# VAE model statement
recons_model = Model(input_img, pred)
optimizer = Adam(learning_rate=LEARNING_RATES[1])
recons_model.compile(optimizer=optimizer, loss='mse')

recons_model.summary()


# # fit and track the model 
fit_model_and_log_metrics("Model recons_1", "With maxpooling, convtranspose, dense, and flatten", recons_model, 50, 128)

## Exp 1.1


In [ ]:
'''
Model recons_1.1
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='leaky_relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='leaky_relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z = Dense(latent_dim)(x)

encoder = Model(input_img,z)

# encoder.summary()

# decoder takes the latent distribution sample as input
# decoder_input = Input(K.int_shape(z)[1:])
# x = Dense(4096, activation='leaky_relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
# x = Reshape((8,8,64))(x)

decoder_input = Input(shape=(latent_dim,))
x = Dense(np.prod(shape_before_flattening[1:]))(decoder_input)
x = Reshape((shape_before_flattening[1], shape_before_flattening[2], shape_before_flattening[3]))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same', activation='leaky_relu')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same')(x)


# decoder model statement
decoder = Model(decoder_input, x)
# decoder.summary()
# apply the decoder to the sample from the latent distribution
pred = decoder(z)

# VAE model statement
recons_model = Model(input_img, pred)
optimizer = Adam(learning_rate=LEARNING_RATES[1])
recons_model.compile(optimizer=optimizer, loss='mse')

recons_model.summary()


# # fit and track the model 
fit_model_and_log_metrics("Model recons_1.1", "With maxpooling, convtranspose, dense, and flatten", recons_model, 50, 128)

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 16, 16, 32)        896       
                                                                 
 batch_normalization_9 (Batc  (None, 16, 16, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_7 (Conv2D)           (None, 8, 8, 16)          4624      
                                                                 
 batch_normalization_10 (Bat  (None, 8, 8, 16)         64        
 chNormalization)                                                
                                                                 
 flatten_3 (Flatten)         (None, 1024)              0   

wandb: Currently logged in as: csc2231-yash-siphelele (use `wandb login --relogin` to force relogin)


Epoch 1/50
391/391 [==============================] - 16s 16ms/step - loss: 0.0322 - val_loss: 0.0317 - _timestamp: 1648858655.0000 - _runtime: 22.0000
Epoch 2/50
391/391 [==============================] - 5s 14ms/step - loss: 0.0066 - val_loss: 0.0061 - _timestamp: 1648858660.0000 - _runtime: 27.0000
Epoch 3/50
391/391 [==============================] - 6s 14ms/step - loss: 0.0048 - val_loss: 0.0047 - _timestamp: 1648858666.0000 - _runtime: 33.0000
Epoch 4/50
391/391 [==============================] - 5s 14ms/step - loss: 0.0040 - val_loss: 0.0036 - _timestamp: 1648858671.0000 - _runtime: 38.0000
Epoch 5/50
391/391 [==============================] - 6s 15ms/step - loss: 0.0037 - val_loss: 0.0036 - _timestamp: 1648858677.0000 - _runtime: 44.0000
Epoch 6/50
391/391 [==============================] - 5s 14ms/step - loss: 0.0033 - val_loss: 0.0034 - _timestamp: 1648858682.0000 - _runtime: 49.0000
Epoch 7/50
391/391 [==============================] - 5s 14ms/step - loss: 0.0032 - val_loss:

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.92754
best_epoch,46
best_val_loss,0.00218
epoch,49
loss,0.00234
val_loss,0.00223


## Exp 1.2

In [ ]:
'''
Model recons_1.2
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='leaky_relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='leaky_relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
print(shape_before_flattening)
x = Flatten()(x)

z = Dense(latent_dim)(x)

encoder = Model(input_img,z)

# encoder.summary()

# decoder takes the latent distribution sample as input
# decoder_input = Input(K.int_shape(z)[1:])
# x = Dense(4096, activation='leaky_relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
# x = Reshape((8,8,64))(x)

decoder_input = Input(shape=(latent_dim,))
x = Dense(np.prod(shape_before_flattening[1:])//2)(decoder_input)
x = Reshape((shape_before_flattening[1], shape_before_flattening[2], shape_before_flattening[3]//2))(x)

x = Conv2DTranspose(16, (3, 3), strides= 2, padding='same', activation='leaky_relu')(x)
x = BatchNormalization()(x)

x = Conv2DTranspose(32, (3, 3), strides= 2, padding='same')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2D(3, (3, 3), padding='same')(x)


# decoder model statement
decoder = Model(decoder_input, x)
# decoder.summary()
# apply the decoder to the sample from the latent distribution
pred = decoder(z)

# VAE model statement
recons_model = Model(input_img, pred)
optimizer = Adam(learning_rate=LEARNING_RATES[1])
recons_model.compile(optimizer=optimizer, loss='mse')

recons_model.summary()


# # fit and track the model 
fit_model_and_log_metrics("Model recons_1.2", "With maxpooling, convtranspose, dense, and flatten", recons_model, 50, 128)

(None, 8, 8, 16)
Model: "model_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_30 (Conv2D)          (None, 16, 16, 32)        896       
                                                                 
 batch_normalization_43 (Bat  (None, 16, 16, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_31 (Conv2D)          (None, 8, 8, 16)          4624      
                                                                 
 batch_normalization_44 (Bat  (None, 8, 8, 16)         64        
 chNormalization)                                                
                                                                 
 flatten_13 (Flatten)        (None, 1024)

Epoch 1/50
391/391 [==============================] - 11s 26ms/step - loss: 0.0407 - val_loss: 0.0284 - _timestamp: 1648859692.0000 - _runtime: 18.0000
Epoch 2/50
391/391 [==============================] - 10s 25ms/step - loss: 0.0082 - val_loss: 0.0069 - _timestamp: 1648859701.0000 - _runtime: 27.0000
Epoch 3/50
391/391 [==============================] - 10s 25ms/step - loss: 0.0060 - val_loss: 0.0057 - _timestamp: 1648859711.0000 - _runtime: 37.0000
Epoch 4/50
391/391 [==============================] - 10s 25ms/step - loss: 0.0051 - val_loss: 0.0051 - _timestamp: 1648859720.0000 - _runtime: 46.0000
Epoch 5/50
391/391 [==============================] - 10s 25ms/step - loss: 0.0045 - val_loss: 0.0041 - _timestamp: 1648859730.0000 - _runtime: 56.0000
Epoch 6/50
391/391 [==============================] - 9s 24ms/step - loss: 0.0041 - val_loss: 0.0048 - _timestamp: 1648859739.0000 - _runtime: 65.0000
Epoch 7/50
391/391 [==============================] - 9s 24ms/step - loss: 0.0039 - val_l

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▂▂▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.92347
best_epoch,49
best_val_loss,0.00229
epoch,49
loss,0.00251
val_loss,0.00229


## Exp 1.3

In [ ]:
'''
Model recons_1.3
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='leaky_relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='leaky_relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
print(shape_before_flattening)
x = Flatten()(x)

z = Dense(512)(x)

encoder = Model(input_img,z)

# encoder.summary()

# decoder takes the latent distribution sample as input
# decoder_input = Input(K.int_shape(z)[1:])
# x = Dense(4096, activation='leaky_relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
# x = Reshape((8,8,64))(x)

decoder_input = Input(shape=(512,))
# x = Dense(np.prod(shape_before_flattening[1:])//2)(decoder_input)
x = Reshape((shape_before_flattening[1], shape_before_flattening[2], shape_before_flattening[3]//2))(decoder_input)

x = Conv2DTranspose(16, (3, 3), strides= 2, padding='same', activation='leaky_relu')(x)
x = BatchNormalization()(x)

x = Conv2DTranspose(32, (3, 3), strides= 2, padding='same')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2D(3, (3, 3), padding='same')(x)


# decoder model statement
decoder = Model(decoder_input, x)
# decoder.summary()
# apply the decoder to the sample from the latent distribution
pred = decoder(z)

# VAE model statement
recons_model = Model(input_img, pred)
optimizer = Adam(learning_rate=LEARNING_RATES[1])
recons_model.compile(optimizer=optimizer, loss='mse')

recons_model.summary()


# # fit and track the model 
fit_model_and_log_metrics("Model recons_1.3", "With maxpooling, convtranspose, dense, and flatten", recons_model, 50, 128)

(None, 8, 8, 16)
Model: "model_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_39 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_45 (Conv2D)          (None, 16, 16, 32)        896       
                                                                 
 batch_normalization_61 (Bat  (None, 16, 16, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_46 (Conv2D)          (None, 8, 8, 16)          4624      
                                                                 
 batch_normalization_62 (Bat  (None, 8, 8, 16)         64        
 chNormalization)                                                
                                                                 
 flatten_19 (Flatten)        (None, 1024)

Epoch 1/50
391/391 [==============================] - 11s 25ms/step - loss: 0.0377 - val_loss: 0.0255 - _timestamp: 1648860739.0000 - _runtime: 16.0000
Epoch 2/50
391/391 [==============================] - 9s 23ms/step - loss: 0.0075 - val_loss: 0.0085 - _timestamp: 1648860748.0000 - _runtime: 25.0000
Epoch 3/50
391/391 [==============================] - 9s 23ms/step - loss: 0.0057 - val_loss: 0.0053 - _timestamp: 1648860757.0000 - _runtime: 34.0000
Epoch 4/50
391/391 [==============================] - 9s 23ms/step - loss: 0.0046 - val_loss: 0.0089 - _timestamp: 1648860767.0000 - _runtime: 44.0000
Epoch 5/50
391/391 [==============================] - 9s 23ms/step - loss: 0.0040 - val_loss: 0.0072 - _timestamp: 1648860776.0000 - _runtime: 53.0000
Epoch 6/50
391/391 [==============================] - 9s 24ms/step - loss: 0.0037 - val_loss: 0.0046 - _timestamp: 1648860785.0000 - _runtime: 62.0000
Epoch 7/50
391/391 [==============================] - 9s 24ms/step - loss: 0.0034 - val_loss:

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▃▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.95165
best_epoch,40
best_val_loss,0.00141
epoch,49
loss,0.00162
val_loss,0.00146


## Exp 1.4

In [ ]:
'''
Model recons_1.4
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='leaky_relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='leaky_relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
print(shape_before_flattening)
x = Flatten()(x)

z = Dense(512)(x)

encoder = Model(input_img,z)

# encoder.summary()

# decoder takes the latent distribution sample as input
# decoder_input = Input(K.int_shape(z)[1:])
# x = Dense(4096, activation='leaky_relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
# x = Reshape((8,8,64))(x)

decoder_input = Input(shape=(512,))
# x = Dense(np.prod(shape_before_flattening[1:])//2)(decoder_input)
x = Reshape((shape_before_flattening[1], shape_before_flattening[2], shape_before_flattening[3]//2))(decoder_input)

x = Conv2DTranspose(32, (3, 3), strides= 2, padding='same', activation='leaky_relu')(x)
x = BatchNormalization()(x)

x = Conv2DTranspose(32, (3, 3), strides= 2, padding='same')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2D(3, (3, 3), padding='same')(x)


# decoder model statement
decoder = Model(decoder_input, x)
# decoder.summary()
# apply the decoder to the sample from the latent distribution
pred = decoder(z)

# VAE model statement
recons_model = Model(input_img, pred)
optimizer = Adam(learning_rate=LEARNING_RATES[1])
recons_model.compile(optimizer=optimizer, loss='mse')

recons_model.summary()


# # fit and track the model 
fit_model_and_log_metrics("Model recons_1.4", "With maxpooling, convtranspose, dense, and flatten", recons_model, 50, 128)

(None, 8, 8, 16)
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 16, 16, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          4624      
                                                                 
 batch_normalization_1 (Batc  (None, 8, 8, 16)         64        
 hNormalization)                                                 
                                                                 
 flatten (Flatten)           (None, 1024) 

wandb: Currently logged in as: csc2231-yash-siphelele (use `wandb login --relogin` to force relogin)


Epoch 1/50
391/391 [==============================] - 20s 27ms/step - loss: 0.0362 - val_loss: 0.0311 - _timestamp: 1649084185.0000 - _runtime: 26.0000
Epoch 2/50
391/391 [==============================] - 9s 24ms/step - loss: 0.0076 - val_loss: 0.0074 - _timestamp: 1649084195.0000 - _runtime: 36.0000
Epoch 3/50
391/391 [==============================] - 10s 24ms/step - loss: 0.0058 - val_loss: 0.0064 - _timestamp: 1649084204.0000 - _runtime: 45.0000
Epoch 4/50
391/391 [==============================] - 11s 29ms/step - loss: 0.0046 - val_loss: 0.0048 - _timestamp: 1649084215.0000 - _runtime: 56.0000
Epoch 5/50
391/391 [==============================] - 12s 30ms/step - loss: 0.0040 - val_loss: 0.0043 - _timestamp: 1649084227.0000 - _runtime: 68.0000
Epoch 6/50
391/391 [==============================] - 10s 24ms/step - loss: 0.0038 - val_loss: 0.0035 - _timestamp: 1649084237.0000 - _runtime: 78.0000
Epoch 7/50
391/391 [==============================] - 10s 24ms/step - loss: 0.0035 - val_

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
loss,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.95126
best_epoch,35
best_val_loss,0.00138
epoch,45
loss,0.00158
val_loss,0.0022


## Exp 1.5

In [ ]:
'''
Model recons_1.5
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='leaky_relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='leaky_relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
print(shape_before_flattening)
x = Flatten()(x)

z = Dense(512)(x)

encoder = Model(input_img,z)

# encoder.summary()

# decoder takes the latent distribution sample as input
# decoder_input = Input(K.int_shape(z)[1:])
# x = Dense(4096, activation='leaky_relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
# x = Reshape((8,8,64))(x)

decoder_input = Input(shape=(512,))
# x = Dense(np.prod(shape_before_flattening[1:])//2)(decoder_input)
x = Reshape((shape_before_flattening[1], shape_before_flattening[2], shape_before_flattening[3]//2))(decoder_input)

x = Conv2DTranspose(24, (3, 3), strides= 2, padding='same', activation='leaky_relu')(x)
x = BatchNormalization()(x)

x = Conv2DTranspose(32, (3, 3), strides= 2, padding='same', activation='leaky_relu')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2D(3, (3, 3), padding='same')(x)


# decoder model statement
decoder = Model(decoder_input, x)
# decoder.summary()
# apply the decoder to the sample from the latent distribution
pred = decoder(z)

# VAE model statement
recons_model = Model(input_img, pred)
optimizer = Adam(learning_rate=LEARNING_RATES[1])
recons_model.compile(optimizer=optimizer, loss='mse')

recons_model.summary()


# # fit and track the model 
fit_model_and_log_metrics("Model recons_1.5", "With maxpooling, convtranspose, dense, and flatten", recons_model, 50, 128)

(None, 8, 8, 16)
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 16, 16, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          4624      
                                                                 
 batch_normalization_1 (Batc  (None, 8, 8, 16)         64        
 hNormalization)                                                 
                                                                 
 flatten (Flatten)           (None, 1024) 

wandb: Currently logged in as: csc2231-yash-siphelele (use `wandb login --relogin` to force relogin)


Epoch 1/50
391/391 [==============================] - 21s 27ms/step - loss: 0.0453 - val_loss: 0.0227 - _timestamp: 1649087911.0000 - _runtime: 27.0000
Epoch 2/50
391/391 [==============================] - 10s 27ms/step - loss: 0.0087 - val_loss: 0.0095 - _timestamp: 1649087921.0000 - _runtime: 37.0000
Epoch 3/50
391/391 [==============================] - 10s 25ms/step - loss: 0.0068 - val_loss: 0.0061 - _timestamp: 1649087931.0000 - _runtime: 47.0000
Epoch 4/50
391/391 [==============================] - 10s 26ms/step - loss: 0.0055 - val_loss: 0.0062 - _timestamp: 1649087941.0000 - _runtime: 57.0000
Epoch 5/50
391/391 [==============================] - 10s 25ms/step - loss: 0.0051 - val_loss: 0.0048 - _timestamp: 1649087951.0000 - _runtime: 67.0000
Epoch 6/50
391/391 [==============================] - 10s 25ms/step - loss: 0.0047 - val_loss: 0.0048 - _timestamp: 1649087961.0000 - _runtime: 77.0000
Epoch 7/50
391/391 [==============================] - 10s 25ms/step - loss: 0.0042 - val

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.95267
best_epoch,43
best_val_loss,0.0014
epoch,49
loss,0.00158
val_loss,0.00141


## Exp 1.6

In [ ]:
'''
Model recons_1.6
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='leaky_relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='leaky_relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
print(shape_before_flattening)
x = Flatten()(x)

z = Dense(1024)(x)

encoder = Model(input_img,z)

# encoder.summary()

# decoder takes the latent distribution sample as input
# decoder_input = Input(K.int_shape(z)[1:])
# x = Dense(4096, activation='leaky_relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
# x = Reshape((8,8,64))(x)

decoder_input = Input(shape=(1024,))
# x = Dense(np.prod(shape_before_flattening[1:])//2)(decoder_input)
x = Reshape((shape_before_flattening[1], shape_before_flattening[2], shape_before_flattening[3]))(decoder_input)

x = Conv2DTranspose(24, (3, 3), strides= 2, padding='same', activation='leaky_relu')(x)
x = BatchNormalization()(x)

x = Conv2DTranspose(32, (3, 3), strides= 2, padding='same', activation='leaky_relu')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2D(3, (3, 3), padding='same')(x)


# decoder model statement
decoder = Model(decoder_input, x)
# decoder.summary()
# apply the decoder to the sample from the latent distribution
pred = decoder(z)

# VAE model statement
recons_model = Model(input_img, pred)
optimizer = Adam(learning_rate=LEARNING_RATES[1])
recons_model.compile(optimizer=optimizer, loss='mse')

recons_model.summary()


# # fit and track the model 
fit_model_and_log_metrics("Model recons_1.6", "With maxpooling, convtranspose, dense, and flatten", recons_model, 50, 128)

## Exp *1.7*

In [ ]:
'''
Model recons_1.7 : 1 skip connection
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='leaky_relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x1 = Conv2D(16, (3, 3), padding='same', strides = 2)(x) # for the skip connection to add later in the decoder
x = LeakyReLU()(x1)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
print(shape_before_flattening)
x = Flatten()(x)

z = Dense(1024)(x)

encoder = Model(input_img,z)

# encoder.summary()

# decoder takes the latent distribution sample as input
# decoder_input = Input(K.int_shape(z)[1:])
# x = Dense(4096, activation='leaky_relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
# x = Reshape((8,8,64))(x)

# decoder_input = Input(shape=(1024,))
x = Dense(np.prod(shape_before_flattening[1:]), activation="leaky_relu")(z)
x = Reshape((shape_before_flattening[1], shape_before_flattening[2], shape_before_flattening[3]))(x)
# x = Reshape((shape_before_flattening[1], shape_before_flattening[2], shape_before_flattening[3]//2))(decoder_input)
x = Conv2DTranspose(16, (3, 3), padding='same')(x)
x = Add()([x1, x])
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Conv2DTranspose(16, (3, 3), strides= 2, padding='same', activation='leaky_relu')(x) # adding the skip connection here
# x = Add()([x1, x])
# x = LeakyReLU()(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(32, (3, 3), strides= 2, padding='same', activation='leaky_relu')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2D(3, (3, 3), padding='same')(x)

pred = x
# decoder model statement
# decoder = Model(decoder_input, x)
# decoder.summary()
# # apply the decoder to the sample from the latent distribution
# pred = decoder(z)

# VAE model statement
recons_model = Model(input_img, pred)
optimizer = Adam(learning_rate=LEARNING_RATES[1])
recons_model.compile(optimizer=optimizer, loss='mse')

recons_model.summary()
 

# # fit and track the model 
fit_model_and_log_metrics("Model recons_1.7", "With maxpooling, convtranspose, dense, and flatten", recons_model, 50, 128)

In [ ]:
# ValueError: Graph disconnected